In [1]:
import requests
import json
import time
import re
import datetime
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_weekday ,var_weekday_name

In [3]:
def eda(data):
    output=[]
    for col in data.columns:
        duplicatedvalue = data[col].duplicated().sum()
        duplicatedrows = data.duplicated().sum()
        missingvalue = np.sum(pd.isna(data[col]))
        uniquevalue = data[col].nunique()
        datatype = str(data[col].dtype)
        
        output.append([col, duplicatedvalue, duplicatedrows, missingvalue, uniquevalue, datatype])
        
    output = pd.DataFrame(output) 
    output.columns = ['Features', 'Duplicated Values', 'Duplicated Rows', 'Missing Values', 'Unique Values', 'Data Type']
    display(output)

In [4]:
def get_last_value(value):
    last = value
    return last

In [5]:
def perc_on_bar(plot, feature):
    total = len(feature)
    for p in ax.patches:
        percentage = "{:.1f}%".format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2 - 0.05
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size=12)
    plt.show()

In [6]:
def viz(data, indice):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title="Neighbor: " + indice + " " + pair + " " + timeframe,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    
    fig.write_image(path + "/" + indice + "_chart.png")
    
    fig.show()

In [7]:
def market_order(instrument, units, take_profit, stop_loss):
    login_request_body = {
        "order": {
            "type": "MARKET",
            "instrument": instrument,
            "units": units,
            "timeInForce": "IOC",
            "positionFill": "DEFAULT",
            "takeProfitOnFill": {
                "price": take_profit
            },
            "stopLossOnFill": {
                "price": stop_loss
            }
        }
    }
    response = requests.post(provider_api_url, data=json.dumps(login_request_body),
                             headers=request_headers,
                             verify=False)
    response
    response.status_code

### The Configs for Run:

In [8]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### <font color='red'>Currency Pair</font>

In [9]:
Load_10K_Records=True


asia = ['EUR_NZD','EUR_AUD','GBP_NZD','GBP_AUD','AUD_USD','AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD']

currency_pairs = ['EUR_USD','EUR_GBP','EUR_NZD','EUR_AUD','EUR_CHF','EUR_CAD',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD','AUD_USD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD',
                  'USD_CAD','USD_CHF','CAD_CHF']

currency_pairs = ["EUR_USD"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/101-001-16477519-001/orders'
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [10]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

### Logging

In [11]:
now = datetime.now()
for pair in currency_pairs:
    Log_Folder = now.strftime("%d-%m-%Y_%I-%M_%p")
    path = os.path.join(Log_Folder+"_"+pair)
    os.mkdir(path)

# Get Candlesticks Data

### Pricing & Spread Cost

In [12]:
for pair in currency_pairs:
    pricing_params = (
        ('instruments', pair),
    )
    response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/pricing'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [13]:
time = response['time']
ask = response['prices'][0]['closeoutAsk']
bid = response['prices'][0]['closeoutBid']
print ('Date:', time, 'Ask:', ask, 'Bid:', bid)

Date: 2021-10-19T03:18:03.614190275Z Ask: 1.16403 Bid: 1.16398


### Open Positions

In [14]:
response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/openPositions'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()
response['positions']

[]

###  Candlestick Data

In [15]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [16]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [17]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [18]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date', 'Time', 'Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [19]:
data.head()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-08-02,21:00:00,6,Sunday,2015-08-02T21:00:00.000000000Z,2197,1.09728,1.09834,1.09656,1.09778
1,2015-08-03,01:00:00,0,Monday,2015-08-03T01:00:00.000000000Z,2186,1.09774,1.09889,1.09708,1.09801
2,2015-08-03,05:00:00,0,Monday,2015-08-03T05:00:00.000000000Z,6207,1.09804,1.09963,1.09574,1.09706
3,2015-08-03,09:00:00,0,Monday,2015-08-03T09:00:00.000000000Z,6964,1.09704,1.09807,1.09412,1.09602
4,2015-08-03,13:00:00,0,Monday,2015-08-03T13:00:00.000000000Z,7750,1.09604,1.09886,1.09457,1.09604


In [20]:
data.tail()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9995,2021-10-18,09:00:00,0,Monday,2021-10-18T09:00:00.000000000Z,23221,1.15809,1.16099,1.15764,1.16015
9996,2021-10-18,13:00:00,0,Monday,2021-10-18T13:00:00.000000000Z,21444,1.16014,1.16219,1.15924,1.16218
9997,2021-10-18,17:00:00,0,Monday,2021-10-18T17:00:00.000000000Z,6954,1.16218,1.16226,1.16086,1.16112
9998,2021-10-18,21:00:00,0,Monday,2021-10-18T21:00:00.000000000Z,2267,1.16102,1.16254,1.16086,1.16251
9999,2021-10-19,01:00:00,1,Tuesday,2021-10-19T01:00:00.000000000Z,4366,1.16252,1.16428,1.16243,1.16400


In [21]:
data = data.drop_duplicates()

In [22]:
data = data.to_csv(filename, header = True, index = False)

In [23]:
data = pd.read_csv(filename)

In [24]:
data.shape

(9660, 10)

# Feature Engineering

### CandleStick Data

In [25]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

#Moving Avarage
data['SMA_10'] = data['Close'].rolling(window=5).mean()
data['SMA_20'] = data['Close'].rolling(window=10).mean()

#Feature Engineering Moving Avarage 
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [26]:
eda(data)

,Features,Duplicated Values,Duplicated Rows,Missing Values,Unique Values,Data Type
0,Date,7721,0,0,1939,object
1,Time,9648,0,0,12,object
2,Weekday,9654,0,0,6,int64
3,Weekday_Name,9654,0,0,6,object
4,UTC_Time,0,0,0,9660,object
5,Volume,2336,0,0,7324,int64
6,Open,2878,0,0,6782,float64
7,High,2810,0,0,6850,float64
8,Low,2857,0,0,6803,float64
9,Close,2825,0,0,6835,float64


In [27]:
data = data.dropna()

In [28]:
data = data.to_csv(filename, header = True, index = False)

In [29]:
data = pd.read_csv(filename)

In [30]:
data.head()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
0,2015-08-04,09:00:00,1,Tuesday,2015-08-04T09:00:00.000000000Z,4688,1.09704,1.09876,1.09588,1.09697,...,0,0.00007,0.00179,0.00116,-0.00275,-0.00099,1.095322,1.096152,0.001648,0.000818
1,2015-08-04,13:00:00,1,Tuesday,2015-08-04T13:00:00.000000000Z,6934,1.09694,1.09816,1.09462,1.09623,...,0,0.00071,0.00193,0.00232,0.00007,-0.00275,1.095564,1.095997,0.000666,0.000233
2,2015-08-04,17:00:00,1,Tuesday,2015-08-04T17:00:00.000000000Z,7139,1.09620,1.09634,1.08792,1.08816,...,0,0.00804,0.00818,0.00828,0.00071,0.00007,1.094542,1.095012,-0.006382,-0.006852
3,2015-08-04,21:00:00,1,Tuesday,2015-08-04T21:00:00.000000000Z,2213,1.08826,1.08940,1.08716,1.08747,...,0,0.00079,0.00193,0.00110,0.00804,0.00071,1.093180,1.094053,-0.005710,-0.006583
4,2015-08-05,01:00:00,2,Wednesday,2015-08-05T01:00:00.000000000Z,2348,1.08744,1.08750,1.08484,1.08678,...,0,0.00066,0.00072,0.00260,0.00079,0.00804,1.091122,1.093129,-0.004342,-0.006349


In [31]:
data.tail()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
9646,2021-10-18,09:00:00,0,Monday,2021-10-18T09:00:00.000000000Z,23221,1.15809,1.16099,1.15764,1.16015,...,1,-0.00206,0.00084,0.00045,-0.00020,0.00054,1.158884,1.159364,0.001266,0.000786
9647,2021-10-18,13:00:00,0,Monday,2021-10-18T13:00:00.000000000Z,21444,1.16014,1.16219,1.15924,1.16218,...,1,-0.00204,0.00001,0.00090,-0.00206,-0.00020,1.159340,1.159644,0.002840,0.002536
9648,2021-10-18,17:00:00,0,Monday,2021-10-18T17:00:00.000000000Z,6954,1.16218,1.16226,1.16086,1.16112,...,0,0.00106,0.00114,0.00132,-0.00204,-0.00206,1.159882,1.159675,0.001238,0.001445
9649,2021-10-18,21:00:00,0,Monday,2021-10-18T21:00:00.000000000Z,2267,1.16102,1.16254,1.16086,1.16251,...,1,-0.00149,0.00003,0.00016,0.00106,-0.00204,1.160808,1.159894,0.001702,0.002616
9650,2021-10-19,01:00:00,1,Tuesday,2021-10-19T01:00:00.000000000Z,4366,1.16252,1.16428,1.16243,1.16400,...,1,-0.00148,0.00028,0.00009,-0.00149,0.00106,1.161992,1.160373,0.002008,0.003627


In [32]:
data.shape

(9651, 26)

# Modeling

In [33]:
data = pd.read_csv(filename)

### Feature Selection and Reduction

In [34]:
data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close',
                          'O-H','O-L','O-C','H-L','H-C','L-C',
                          'SMA_10',
                          'SMA_20',
#                          'col_1',
#                          'col_2',
#                          'col_3',
#                          'col_4',
#                          'col_5',
#                          'F_SMA_10',
#                          'F_SMA_20'
                         ])

#'Volume', 'Direction'

In [35]:
data.head()

,col_1,col_2,col_3,col_4,col_5,F_SMA_10,F_SMA_20
0,0.00007,0.00179,0.00116,-0.00275,-0.00099,0.001648,0.000818
1,0.00071,0.00193,0.00232,0.00007,-0.00275,0.000666,0.000233
2,0.00804,0.00818,0.00828,0.00071,0.00007,-0.006382,-0.006852
3,0.00079,0.00193,0.00110,0.00804,0.00071,-0.005710,-0.006583
4,0.00066,0.00072,0.00260,0.00079,0.00804,-0.004342,-0.006349


In [36]:
data.tail()

,col_1,col_2,col_3,col_4,col_5,F_SMA_10,F_SMA_20
9646,-0.00206,0.00084,0.00045,-0.00020,0.00054,0.001266,0.000786
9647,-0.00204,0.00001,0.00090,-0.00206,-0.00020,0.002840,0.002536
9648,0.00106,0.00114,0.00132,-0.00204,-0.00206,0.001238,0.001445
9649,-0.00149,0.00003,0.00016,0.00106,-0.00204,0.001702,0.002616
9650,-0.00148,0.00028,0.00009,-0.00149,0.00106,0.002008,0.003627


# <font color='red'>CANDLE INDEX NUMBER</font>

In [37]:
candle_no = len(data) - 2
candle_no

9649

### Scaling using Standard Scaler

In [38]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'brute') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
            #print("Recommendation for {0}:\n".format(eurusd_data.index[candle_id]))
        else:
            #print("{0}: {1}, with distance of {2}".format(i,
            #                                               dataset.index[indices.flatten()[i]],
            #                                               distances.flatten()[i]))
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
 #                           dataset.iloc[indices.flatten()[i]]['O-H'],
 #                           dataset.iloc[indices.flatten()[i]]['O-L'],
 #                           dataset.iloc[indices.flatten()[i]]['O-C'],
 #                           dataset.iloc[indices.flatten()[i]]['H-L'],
 #                           dataset.iloc[indices.flatten()[i]]['H-C'],
 #                           dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],
                           dataset.iloc[indices.flatten()[i]]['col_2'],
                           dataset.iloc[indices.flatten()[i]]['col_3'],
                           dataset.iloc[indices.flatten()[i]]['col_4'],
                           dataset.iloc[indices.flatten()[i]]['col_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_10'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      'col_1',
                      'col_2',
                      'col_3',
                      'col_4',
                      'col_5',
                      'F_SMA_10',
                      'F_SMA_20']
    display (output)
    
    return indices, distances

### Top 5 Similar Candlesticks

In [39]:
indices, distances = find_k_similar_candles (candle_no,data)

,col_1,col_2,col_3,col_4,col_5,F_SMA_10,F_SMA_20
9649,-0.00149,0.00003,0.00016,0.00106,-0.00204,0.001702,0.002616


,Indice,Distance,col_1,col_2,col_3,col_4,col_5,F_SMA_10,F_SMA_20
0,8987,0.000571,-0.00162,0.00017,0.00018,0.00080,-0.00183,0.001440,0.002287
1,4169,0.000860,-0.00154,0.00054,0.00007,0.00086,-0.00243,0.001820,0.003128
2,8124,0.000951,-0.00178,0.00038,0.00010,0.00097,-0.00148,0.002154,0.003026


In [40]:
indices = indices[0:1][0]

In [41]:
indices

array([9649, 8987, 4169, 8124])

### Currnet Market/Candlestick

In [45]:
closed_candle = "currnet_market_data.csv"
data = pd.read_csv(filename)
data = data.iloc[candle_no-50:candle_no+2]
data.to_csv(path + "/" + closed_candle, header = True, index = False)
viz(data, "current_market")

### Recommendations

In [47]:
for indice in indices[1:6]:
    recommendation_log = "{}_data.csv".format(indice)
    data = pd.read_csv(filename)
    data = data.iloc[indice-2:indice+50]
    data.to_csv(path + "/" + recommendation_log, header = True, index = False)
    print ('Neighbor:', indice, '|', '10K Records:', Load_10K_Records)
    viz(data, indice.astype(str))

Neighbor: 8987 | 10K Records: True


Neighbor: 4169 | 10K Records: True


Neighbor: 8124 | 10K Records: True


In [44]:
currentpath = os.path.join(sys.path[0])
pngfiles = []
pngfiles = glob.glob(currentpath+"/"+path+"/*.png")
pdf = FPDF()

for pngfile in pngfiles:
    pdf.add_page()
    pdf.image(pngfile, w=200, h=130)
    
pdf.output(path+"/recommendations.pdf", "F")

''